In [1]:
# Importar librerías 
import DataFunctions
import ModelFunctions
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from valuation import Companies, Models
from sklearn.impute import KNNImputer
import statsmodels.api as sm
import numpy as np
import pandas as pd
#from ydata_profiling import ProfileReport

In [2]:
# Cargar Datos
income_statement = DataFunctions.load_full_excel('Data/Income/Income_Statement.xlsx')
balance_statement = DataFunctions.load_full_excel('Data/Balance/Balance_Statement.xlsx')
sp500 = DataFunctions.assets(income_statement=income_statement)

# Si se desea cambiar de periodo de analisis cambiar la siguiente fecha:
prices = yf.download(tickers=sp500, start='2018-09-01', end='2023-09-23', progress=False)['Adj Close'] 

# Filtrado de fechas y correcciones temporales
prices_fiscal = DataFunctions.prices_date(balance_statement=balance_statement, prices=prices, sp500=sp500)

# DataFrame limpio con información consolidada de ambos estados financieros y precios
financial_info = DataFunctions.clean_df(balance_statement=balance_statement, 
                                    income_statement=income_statement, 
                                    sp500=sp500, 
                                    prices_fiscal=prices_fiscal)

# Datos en formato tabular
data_table = DataFunctions.tabular_df(financial_info=financial_info, sp500=sp500)

# Cálculo de ratios financieros 
stock = Companies(data_table)
stock.get_ratios()

# EDA con miras a limpieza de datos
DataFunctions.dqr(stock.ratios)

# Limpieza
stock.clean_ratios()
stock.clean


1 Failed download:
['ABC']: Exception('%ticker%: No data found, symbol may be delisted')


,Stock,fiscalDateEnding,PER,PBV,Acid_test,ATR,CCC,ROA,DER,NPM,EM,Return
0,A,2023-06-30,117.226398,3.280427,26.970165,0.156505,484.824339,0.027984,0.866805,0.178804,1.866805,1
1,A,2023-03-31,115.687540,3.729464,12.466387,0.159538,473.104821,0.032237,0.946693,0.202067,1.946693,1
2,A,2022-12-31,119.300066,4.168479,76.111111,0.174041,425.001250,0.034941,0.985297,0.200764,1.985297,1
3,A,2022-09-30,108.403270,3.401820,14.888889,0.163201,419.650125,0.031381,1.059320,0.192285,2.059320,0
4,A,2022-06-30,128.247944,3.361065,15.462857,0.151698,416.910303,0.026208,1.041195,0.172762,2.041195,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14,XRAY,2019-12-31,197.198231,1.773365,4.074675,0.101090,520.351878,0.008993,0.990733,0.088959,1.990733,1
15,XRAY,2019-09-30,434.443191,1.833476,26.970165,0.116104,445.294066,0.004220,0.816937,0.036349,1.816937,0
16,XRAY,2019-06-30,450.749793,1.958478,26.970165,0.104921,447.986379,0.004345,0.760390,0.041411,1.760390,1
17,XRAY,2019-03-31,8354.719810,1.679903,26.970165,0.118376,407.148055,0.000201,0.777601,0.001699,1.777601,0


In [3]:
import joblib
from locale import setlocale, LC_TIME

database = stock.clean.copy()
loaded_model = joblib.load(open('GBC_bagging_model.pkl', 'rb'))

setlocale(LC_TIME, 'en_US.UTF-8')

# Model
#database.to_csv('data.csv')
data = pd.read_csv('data.csv').drop('Unnamed: 0', axis=1)
data['fiscalDateEnding'] = pd.to_datetime(data['fiscalDateEnding'], format='%Y-%m-%d')
data['Yhat'] = loaded_model.predict(data.drop(['Stock', 'fiscalDateEnding', 'Return'], axis=1))
data = data.sort_values(by = 'fiscalDateEnding', ascending = True)

# Prices
prices = stock.df[['Stock','fiscalDateEnding','Adj Close']]
prices = prices.sort_values(by = 'fiscalDateEnding')

data = data.merge(prices, how = 'left', on = ['fiscalDateEnding','Stock'])
data.head()

,Stock,fiscalDateEnding,PER,PBV,Acid_test,ATR,CCC,ROA,DER,NPM,EM,Return,Yhat,Adj Close
0,GEN,2018-12-31,-397.673958,0.371745,26.970165,0.071512,906.206897,-0.000935,0.862399,-0.013072,1.86327,0,0,9.880100
1,DISH,2018-12-31,18.623365,0.939512,41.532332,0.396907,209.470823,0.050448,0.862399,0.127103,1.86327,0,0,24.969999
2,DLTR,2018-12-31,103.204542,3.398345,26.970165,0.647207,20.407688,0.032928,0.862399,0.050877,1.86327,0,0,90.320000
3,DOV,2018-12-31,134.515580,2.472537,26.970165,0.204184,95.486986,0.018381,0.862399,0.090022,1.86327,0,0,65.714188
4,DOW,2018-12-31,0.000000,0.000000,26.970165,1.740944,8.315140,0.156812,0.862399,0.090073,1.86327,0,0,0.000000


In [4]:
rf = pd.read_csv("^IRX.csv")
rf.Date = pd.to_datetime(rf['Date'], format='%m/%d/%y')

rf["rf"] = rf["Adj Close"]
rf.drop(["Adj Close"],axis=1 ,inplace=True)

rf["fiscalDateEnding"] = rf.Date
rf.drop(["Date"],axis=1 ,inplace=True)

rf.fillna(method='ffill', inplace=True)

data = data.merge(rf, on="fiscalDateEnding")

investment_rf = {0: 1, 
                 1: 0.8, 
                 2: 0.6, 
                 3: 0.4, 
                 4: 0.2, 
                 5: 0}

initial_capital = 1000000
comision = .00025

#date = '2018-12-31'
date = '2019-06-30'
trade_period = data[data['fiscalDateEnding'] == date]

In [5]:
import AssetAllocation as AA

In [6]:
# ASSET PICKER
def pick_assets(data: pd.DataFrame, assets: pd.DataFrame, fiscal_date: str):
    # Previous Stocks Evaluation
    previous_stocks = data[(data['fiscalDateEnding'] == fiscal_date) & (data['Yhat'] == 1)].merge(assets, on= 'Stock', how = 'inner')
    previous_stocks = previous_stocks['Stock'].values
    # Add Missing Stocks
    new_assets = 5 - len(previous_stocks)
    try:
        add_assets = data[(data['fiscalDateEnding'] == fiscal_date) & (data['Yhat'] == 1)].sample(n = new_assets)['Stock'].values
        assets_list = list(previous_stocks) + list(add_assets) 
    except:
        assets_list = []
    return assets_list



In [7]:
# OMEGA ASSET ALLOCATION
def omegaAA(data: pd.DataFrame, assets: pd.DataFrame, assets_lists: list, fiscal_date: str, mkt_idx: str = '^GSPC'):
    # Omega Optimization
    rf_rate = data.rf.values[0] / 100
    if len(assets_lists) > 0:
        tickers = assets_lists.copy()
        tickers.append(mkt_idx)
        end_date = pd.to_datetime(fiscal_date)
        start_date = end_date + timedelta(days = -365)
        try:
            omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        except:
            try:
                omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
            except:
                try:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
                except:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        omega = AA.asset_allocation(data_stocks=omega_prices[omega_prices.columns[:-1]], data_benchmark=omega_prices[omega_prices.columns[-1]].to_frame(), rf=rf_rate)
        omega_weights = omega.omega(n_port=1)
    else:
        omega_weights = []
    # RF
    rf_percentage = (5 - len(omega_weights)) * .2
    if rf_percentage > 0:
        assets_lists = assets_lists + ['Rf']
        omega_weights = np.array(omega_weights) * (1 - rf_percentage)
        omega_weights = np.concatenate((omega_weights, [rf_percentage]))
        omega_weights = list(omega_weights).copy()

    # Assets DF
    new_assets = pd.DataFrame([ pd.to_datetime([fiscal_date for i in range(len(omega_weights))]),
                             assets_lists, omega_weights],
                           index=['Date','Stock','W']).transpose()
    assets = pd.concat([assets, new_assets], axis = 0, ignore_index = True)

    # Omega Weights
    omega_weights = {asset:[weight] for asset,weight in zip(assets_lists,omega_weights)}
    omega_weights = pd.DataFrame.from_dict(omega_weights).T.reset_index().rename(columns = {'index':'Stock',0:'Weight'})

    return omega_weights, assets

In [11]:
## Init
current_open_positions = pd.DataFrame(columns=['Date','Asset','X','Price'])
operations = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])
cash = 10**6

In [52]:
## Sell RF
def open_positions(current_open_positions: pd.DataFrame, consult_asset: str):
    return consult_asset in  current_open_positions.Asset.values

def valuate_position(current_open_positions: pd.DataFrame, consult_asset: str):
    rf_open_position = current_open_positions[current_open_positions['Asset'] == consult_asset]['X'].values[0]        # Verify Work
    position_date = current_open_positions[current_open_positions['Asset'] == consult_asset]['Date'].values[0]        # Verify Work
    return   rf_open_position, position_date

def close_position(current_open_positions: pd.DataFrame):
    current_open_positions = current_open_positions[current_open_positions['Asset'] != 'Rf']
    return current_open_positions

def report_sale(operations: pd.DataFrame, rf_open_position: float, position_date: str, fiscal_date: str):
    previous_rf = operations[(operations['Date'] == position_date) & (operations['Stock'] == 'Rf') & (operations['Type'] == 'Buy')]['Price'].values[0] / 100
    income_risk_free =  rf_open_position * ( 1 + previous_rf *  3/12)
    sell_operation = pd.DataFrame([fiscal_date,'Rf',income_risk_free,previous_rf,-income_risk_free,'Sell'], index = ['Date','Stock','X','Price','Position','Type']).T
    operations = pd.concat([operations,sell_operation], axis = 1, ignore_index = True)
    return operations, income_risk_free

def update_funds(income_risk_free: float):
    cash = income_risk_free
    return cash
    
def sell_rf(current_open_positions: pd.DataFrame, operations: pd.DataFrame, fiscal_date: str):
    if open_positions(current_open_positions, 'Rf'):
        rf_open_position, position_date = valuate_position(current_open_positions, 'Rf')
        current_open_positions = close_position(current_open_positions)
        operations, income_risk_free = report_sale(operations, rf_open_position, position_date, fiscal_date)
        cash = update_funds(income_risk_free)
    else:
        income_risk_free = 0
        cash = update_funds(income_risk_free)
    return current_open_positions, operations, cash

In [263]:
## Current Port Value
def previous_capitals_open(current_open_positions: pd.DataFrame):
    return len(current_open_positions[current_open_positions['Asset'] != 'Rf']['Asset'].values) > 0

def get_market_prices(tickers, start_date: str, end_date: str):
    try:
        prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    except:
        try:
            prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        except:
            try:
                prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
            except:
                prices_new = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
    prices_new = prices_new.iloc[0]
    prices_new = prices_new.to_frame()
    prices_new.columns = ['Price']
    prices_new = prices_new.reset_index().rename(columns = {'index':'Stock'}).set_index('Stock')
    return prices_new

def current_position_value(current_open_positions: pd.DataFrame, fiscal_date: str):
    tickers_previous = list(current_open_positions[current_open_positions['Asset'] != 'Rf']['Asset'].values)
    end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
    end_date = end_date.strftime('%Y-%m-%d')

    previous_portfolio = current_open_positions[['Asset','X','Price']].set_index('Asset')
    prices_new = get_market_prices(tickers_previous, start_date=fiscal_date, end_date= end_date).iloc[0]
    
    previous_portfolio = previous_portfolio.merge(prices_new, left_index = True, right_index = True, how = 'left')
    previous_portfolio['Pos'] = previous_portfolio['X'] * previous_portfolio['Price']
    previous_portfolio = previous_portfolio.rename(columns = {'X':'X_1'})
    
    capitals_value = previous_portfolio['Pos'].sum()
    return previous_portfolio, capitals_value

def overall_portfolio_value(capitals_value: float, cash: float):
    return capitals_value + cash

def no_previous_capital_open():
    return pd.DataFrame(columns = ['Stock','X_1']).set_index('Stock')

def current_portfolio_value(current_open_positions: pd.DataFrame, cash: float, fiscal_date: str):
    if previous_capitals_open(current_open_positions):
        previous_portfolio, capitals_value = current_position_value(current_open_positions, fiscal_date)
        cash = overall_portfolio_value(capitals_value, cash)
    else:
        previous_portfolio = no_previous_capital_open()
        cash = cash # No aditional cash from capitals
    return previous_portfolio, cash

In [279]:
## Trade Capitals
def adjustments_to_portofolio(omega_weights: pd.DataFrame, previous_portfolio: pd.DataFrame, cash: float, fiscal_date: str, commission: float = .00025):
    Xt = omega_weights[omega_weights['Stock'] != 'Rf'].copy()
    end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
    end_date = end_date.strftime('%Y-%m-%d')
    tickers = list(Xt['Stock'].values)

    try:
        prices_new = get_market_prices(tickers=tickers, start_date=fiscal_date, end_date= end_date)
    except:
        prices_new = pd.DataFrame(columns = ['Price'])
    

    Xt = Xt.set_index('Stock')
    Xt = Xt.merge(prices_new, left_index = True, right_index = True, how = 'left') 
    #Xt = pd.concat([Xt,prices_new], axis = 1)

    Xt["X"] = Xt['Weight'] * cash / Xt['Price']
    Xt = Xt[['X','Price']]

    # X
    X = Xt.join(previous_portfolio['X_1'], how='outer').fillna(0.0)
    #X = pd.concat([Xt, previous_portfolio], axis = 1).fillna(0.0)
    X['Trade'] = X['X'] - X['X_1']

    # Only capitals
    X = X.reset_index().rename(columns = {'Stock':'Asset'})
    print(X)

    # Sell 
    to_sell = X['Trade']<0
    if len(X[to_sell]) > 0:
        assets_to_sell = np.ceil((X[to_sell].Trade) * (1+commission))
        cash_gain_per_asset = assets_to_sell * X[to_sell]['Price'] * (1-commission)
        operations_sell = assets_to_sell.to_frame().rename(columns = {'Trade':'X'})
        operations_sell['Price'] = X[to_sell]['Price'].values
        operations_sell['Asset'] = X[to_sell]['Asset'].values
        operations_sell['Position'] = cash_gain_per_asset.T.values
        operations_sell['Type'] = 'Sell'
    else:
        #net_cash_gain = 0
        operations_sell = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])
    
    # Buy
    to_buy = X['Trade']>0
    if len(X[to_buy]) > 0:
        assets_to_buy = np.floor((X[to_buy].Trade) * (1-commission))
        cash_invest_per_asset = assets_to_buy * X[to_buy]['Price'] * (1+commission)
        #net_cash_invest = cash_invest_per_asset.sum(axis=1)
        operations_buy = assets_to_buy.to_frame().rename(columns = {'Trade':'X'})
        operations_buy['Price'] = X[to_buy]['Price'].values
        operations_buy['Asset'] = X[to_buy]['Asset'].values
        operations_buy['Position'] = cash_invest_per_asset.T.values
        operations_buy['Type'] = 'Buy'
    else:
        operations_buy = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])

    

    new_operations = pd.concat([operations_sell,operations_buy], axis = 0, ignore_index = True)
    return new_operations

def close_capitals_position(current_open_positions: pd.DataFrame, new_operations: pd.DataFrame):
    sell_operations = new_operations[new_operations['Type'] == 'Sell']
    amount_open = current_open_positions[['Asset','X','Price']]
    amount_to_close = sell_operations[['Date','Stock','X','Price']]

    for stock in amount_to_close.Stock:
        if stock in amount_open.Asset:
            asset_open = amount_open[amount_open.Asset == stock]
            asset_to_close = amount_to_close[amount_to_close.Stock == stock]
            remaining_open = asset_open.X - asset_to_close.X
            if remaining_open > 0:
                price = asset_open.Price
                new_open_date = asset_to_close.Date
                new_entry = pd.DataFrame([new_open_date, stock, remaining_open, price], index=['Date','Asset','X','Price']).T # FIX
                current_open_positions = pd.concat([current_open_positions, new_entry], axis=0, ignore_index=True)
            else:
                current_open_positions = current_open_positions[current_open_positions['Asset'] != stock]
        else:
            pass
    return current_open_positions

def open_capitals_position(current_open_positions: pd.DataFrame, new_operations: pd.DataFrame):
    buy_operations = new_operations[new_operations['Type'] == 'Buy']
    amount_open = current_open_positions[['Asset','X','Price']]
    amount_to_open = buy_operations[['Date','Stock','X','Price']]

    for stock in amount_to_open.Stock:
        if stock in amount_open.Asset:
            asset_open = amount_open[amount_open.Asset == stock]
            asset_to_open = amount_to_open[amount_to_open.Stock == stock]
            remaining_open = asset_open.X + asset_to_open.X
            price = asset_open.Price * ( asset_open / remaining_open ) + asset_to_open.Price * ( asset_to_open.X / remaining_open )
            new_open_date = asset_to_open.Date
            new_entry = pd.DataFrame([new_open_date, stock, remaining_open, price], index=['Date','Asset','X','Price']).T
            current_open_positions = pd.concat([current_open_positions, new_entry], axis=0, ignore_index=True)
        else:
            asset_open = amount_open[amount_open.Asset == stock]
            asset_to_open = amount_to_open[amount_to_open.Stock == stock]
            remaining_open = asset_to_open.X
            price = asset_to_open.Price
            new_open_date = asset_to_open.Date
            new_entry = pd.DataFrame([new_open_date, stock, remaining_open, price], index=['Date','Asset','X','Price']).T
            current_open_positions = pd.concat([current_open_positions, new_entry], axis=0, ignore_index=True)

    return current_open_positions

def open_capitals_position(current_open_positions: pd.DataFrame, new_operations: pd.DataFrame):  # CHAT GPT
    buy_operations = new_operations[new_operations['Type'] == 'Buy']
    amount_open = current_open_positions[['Asset', 'X', 'Price']]
    amount_to_open = buy_operations[['Date', 'Stock', 'X', 'Price']]

    for stock in amount_to_open['Stock'].values:
        if stock in amount_open['Asset'].values:
            asset_open = amount_open[amount_open['Asset'] == stock]
            asset_to_open = amount_to_open[amount_to_open['Stock'] == stock]
            remaining_open = asset_open['X'] + asset_to_open['X']
            price = (asset_open['Price'].iloc[0] * asset_open['X'].iloc[0] + asset_to_open['Price'].iloc[0] * asset_to_open['X'].iloc[0]) / remaining_open
            new_open_date = asset_to_open['Date']
            new_entry = pd.DataFrame([[new_open_date, stock, remaining_open, price]], columns=['Date', 'Asset', 'X', 'Price'])
            current_open_positions = pd.concat([current_open_positions, new_entry], ignore_index=True)
        else:
            asset_to_open = amount_to_open[amount_to_open['Stock'] == stock]
            remaining_open = asset_to_open['X']
            price = asset_to_open['Price']
            new_open_date = asset_to_open['Date']
            new_entry = pd.DataFrame([[new_open_date, stock, remaining_open, price]], columns=['Date', 'Asset', 'X', 'Price'])
            current_open_positions = pd.concat([current_open_positions, new_entry], ignore_index=True)

    return current_open_positions


def report_operations(operations: pd.DataFrame, new_operations: pd.DataFrame):
    return pd.concat([operations, new_operations], axis=0, ignore_index=True)

def trade_capitals(current_open_positions: pd.DataFrame, operations: pd.DataFrame, omega_weights:pd.DataFrame, 
                   previous_portfolio: pd.DataFrame, cash: float, fiscal_date: pd.DataFrame):
    
    new_operations = adjustments_to_portofolio(omega_weights, previous_portfolio, cash, fiscal_date)
    current_open_positions = close_capitals_position(current_open_positions, new_operations)
    current_open_positions = open_capitals_position(current_open_positions, new_operations)
    operations = report_operations(operations, new_operations)
    return current_open_positions, operations, new_operations
        


In [229]:
## Buy RF
cash

1000000

In [264]:
# Trade Function
def TradeCapitals(current_open_positions: pd.DataFrame, operations: pd.DataFrame, omega_weights: pd.DataFrame, fiscal_date: str,initial_cycle: bool = False ,initial_capital: int = 1000000):
    current_open_positions, operations, cash = sell_rf(current_open_positions, operations, fiscal_date)
    if initial_cycle:
        cash = initial_capital
    previous_portfolio, cash = current_portfolio_value(current_open_positions, cash, fiscal_date)
    current_open_positions, operations, new_operations = trade_capitals(current_open_positions, operations, omega_weights, previous_portfolio, cash, fiscal_date)
    #buy_rf()
    #exercise_report()
    return current_open_positions, operations, new_operations

In [ ]:
## Init
current_open_positions = pd.DataFrame(columns=['Date','Asset','X','Price'])
operations = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])
cash = 10**6

In [ ]:

def trade(data: pd.DataFrame, assets: pd.DataFrame, operations: pd.DataFrame, positions_status_rf: pd.DataFrame, omega_weights: pd.DataFrame, portfolio_valuation: pd.DataFrame, fiscal_date: str, initial_capital: int = 1000000, commission: float = 0.00025):   
    #omega_weights
    omega_weights = omega_weights[omega_weights['Stock'] != 'Rf']['Weight'].values
    assets_rf = assets[assets['Stock'] == 'Rf'].copy()
    previous_rf_date = assets_rf[assets_rf['Date'] != fiscal_date]['Date'].max()

    try:
        previous_rf_date = previous_rf_date.strftime('%Y-%m-%d')
    except:
        pass

    # Sell Previous Position in Rf
    try:
        bought_rf = operations[(operations['Date'] == previous_rf_date) & (operations['Stock'] == 'Rf') & (operations['Type'] == 'Buy')]
        closed_position = positions_status_rf[
            (positions_status_rf['Rate'] == bought_rf['Price'].values[0]) & (positions_status_rf['Position'] == bought_rf['Position'].values[0])
            ]['Status'].values
    except:
        pass

    if len(bought_rf) > 0 and ('Closed' not in closed_position):
        print('')
        print('Eval: ',len(bought_rf) > 0,' , ','Closed' not in closed_position)
        rf_1 = bought_rf['Price'].values[0]
        rf_position = bought_rf['Position'].values[0]
        income_risk_free =  rf_position * ( 1 + rf_1 *  3/12)
        operations_sell_rf = pd.DataFrame([fiscal_date,'Rf',income_risk_free,rf_1,-income_risk_free,'Sell'], index = ['Date','Stock','X','Price','Position','Type']).T
        close = pd.DataFrame([fiscal_date,rf_1,income_risk_free,'Closed'], index = ['Date','Rate','Position','Status']).T
        positions_status_rf = pd.concat([positions_status_rf,close], axis = 0, ignore_index = True)
        print('Close position')
    else:
         operations_sell_rf = pd.DataFrame(columns = ['Date','Stock','X','Price','Position','Type'])
         income_risk_free = 0
    

    # Capitals Trade
    # Data
    assets = assets[assets['Stock'] != 'Rf'].copy()
    tickers_new = assets[assets['Date'] == fiscal_date]['Stock'].values
    previous_date = assets[assets['Date'] != fiscal_date]['Date'].max()

    try:
        previous_date = previous_date.strftime('%Y-%m-%d')
    except:
        pass

    if len(assets) > 0:
        tickers_previous = assets[assets['Date'] == previous_date]['Stock'].values
        end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
        tickers = np.unique(np.array(list(tickers_new) + list(tickers_previous)))
        tickers = list(tickers)
        try:
            prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
        except:
            try:
                prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
            except:
                try:
                    prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
                except:
                    prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
        prices_new = prices_new.to_frame()
        prices_new.columns = ['Price']
    else:
        prices_new = pd.DataFrame(columns=['Price','Stock']).set_index('Stock')

    # X_T-1
    Xt_prior = operations[(operations['Stock'] != 'Rf') & (operations['Date'] == previous_date)].set_index('Stock')['X'].to_frame()
    Xt_prior = Xt_prior.merge(prices_new, left_index = True, right_index = True, how = 'left')
    Xt_prior['Pos'] = Xt_prior['X'] * Xt_prior['Price']
    Val_Port = Xt_prior['Pos'].sum() # income_risk_free
    print('Capitals:',Xt_prior['Pos'].sum(),' RF: ', income_risk_free)
    if Val_Port == 0:
        Val_Port = initial_capital
    Xt_prior = Xt_prior.rename(columns = {'X':'X_1'})
    ## Valuation
    # X_T 
    Xt = pd.DataFrame(omega_weights, index = tickers_new, columns=['W'])
    Xt = Xt.merge(prices_new, left_index = True, right_index = True, how = 'left') 
    Xt["X"] = Xt['W'] * Val_Port / Xt['Price']
    Xt = Xt['X'].to_frame()

    # X
    X = Xt.join(Xt_prior, how='outer').fillna(0.0)
    X['Trade'] = X['X'] - X['X_1']

    # Only capitals
    X = X.reset_index().rename( columns = {'index':'Stock'})
    X = X[X['Stock'] != 'Rf'].set_index('Stock') # Posible Redundancia

    
    # Sell 
    to_sell = X['Trade']<0
    if len(X[to_sell]) > 0:
        assets_to_sell = np.ceil((X[to_sell].Trade) * (1+commission))
        cash_gain_per_asset = assets_to_sell * prices_new[to_sell].T * (1-commission)
        #net_cash_gain = cash_gain_per_asset.sum(axis=1)
        operations_sell = assets_to_sell.to_frame().rename(columns = {'Trade':'X'})
        operations_sell['Price'] = prices_new[to_sell].values
        operations_sell['Stock'] = X[to_sell].index.values
        operations_sell['Position'] = cash_gain_per_asset.T.values
        operations_sell['Type'] = 'Sell'
    else:
        #net_cash_gain = 0
        operations_sell = pd.DataFrame(columns=['X','Price','Stock','Position','Type'])
    
    # Buy
    to_buy = X['Trade']>0
    if len(X[to_buy]) > 0:
        assets_to_buy = np.floor((X[to_buy].Trade) * (1-commission))
        cash_invest_per_asset = assets_to_buy * prices_new[to_buy].T * (1+commission)
        #net_cash_invest = cash_invest_per_asset.sum(axis=1)
        operations_buy = assets_to_buy.to_frame().rename(columns = {'Trade':'X'})
        operations_buy['Price'] = prices_new[to_buy].values
        operations_buy['Stock'] = X[to_buy].index.values
        operations_buy['Position'] = cash_invest_per_asset.T.values
        operations_buy['Type'] = 'Buy'
    else:
        #net_cash_invest = 0
        operations_buy = pd.DataFrame(columns=['X','Price','Stock','Position','Type'])

    
    # Buy Risk Free
    current_date_rf_assets = assets_rf[assets_rf['Date'] == fiscal_date]
    if len(current_date_rf_assets) > 0:
        weight_rf = current_date_rf_assets['W'].values[0]
        rf = data[data["fiscalDateEnding"]==fiscal_date].rf.values[0] / 100
        buy_risk_free = Val_Port*weight_rf
        operations_buy_rf = pd.DataFrame([fiscal_date,'Rf',buy_risk_free,rf,buy_risk_free,'Buy'], index = ['Date','Stock','X','Price','Position','Type']).T
        open_pos = pd.DataFrame([fiscal_date,rf,buy_risk_free,'Open'], index = ['Date','Rate','Position','Status']).T
        positions_status_rf = pd.concat([positions_status_rf,open_pos], axis = 0, ignore_index = True)
    else:
         operations_buy_rf = pd.DataFrame(columns = ['Date','Stock','X','Price','Position','Type'])
         buy_risk_free = 0
    # Operations
    operations_new = pd.concat([operations_sell,operations_buy], axis = 0)
    operations_new['Date'] = fiscal_date
    operations_new = operations_new[['Date','Stock','X','Price','Position','Type']]
    operations = pd.concat([operations,operations_new,operations_buy_rf,operations_sell_rf], axis = 0, ignore_index=True)

    # Cash
    #net_cash_remainer = net_cash_gain + income_risk_free - net_cash_invest - buy_risk_free # Redundancia
    #cash = initial_capital - net_cash_remainer # Redundancia
    
    # Portfolio Value
    portfolio_value = pd.DataFrame([fiscal_date,operations[
        (operations['Date'] == fiscal_date) & (operations['Type'] == 'Buy')
        ]['Position'].sum()], index = ['Date','Value_after_commissions']).T
    portfolio_valuation = pd.concat([portfolio_valuation,portfolio_value], axis = 0, ignore_index=True)

    return operations, portfolio_valuation, positions_status_rf, assets_rf

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [280]:
assets = pd.DataFrame(columns=['Date','Stock','W'])
current_open_positions = pd.DataFrame(columns=['Date','Asset','X','Price'])
portfolio_valuation = pd.DataFrame(columns=['Date','Value_after_commissions'])
operations = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])
first_date = data['fiscalDateEnding'].unique()[2].strftime('%Y-%m-%d')
wo_abc = data[data['Stock'] != 'ABC']

for fiscal_date in data['fiscalDateEnding'].unique()[2:5]:
    print(fiscal_date)
    fiscal_date = fiscal_date.strftime('%Y-%m-%d')
    assets_list = pick_assets(data=wo_abc, assets=assets, fiscal_date=fiscal_date)
    omega_weights, assets = omegaAA(data=wo_abc, assets=assets, assets_lists=assets_list, fiscal_date=fiscal_date)
    if fiscal_date == first_date:
        current_open_positions, operations, new_operations = TradeCapitals(current_open_positions, operations, omega_weights,
                                                                    fiscal_date, True)
    else:
        current_open_positions, operations, new_operations = TradeCapitals(current_open_positions, operations, omega_weights, fiscal_date)
        

2019-06-30 00:00:00
  Asset             X       Price  X_1         Trade
0    CI   1459.064726  151.330109  0.0   1459.064726
1   HPQ   3441.201887   18.286968  0.0   3441.201887
2  AMCR  27936.226236    9.518432  0.0  27936.226236
3   HPE  23895.271696   13.088141  0.0  23895.271696
4   MOS   5758.471533   23.898125  0.0   5758.471533
2019-09-30 00:00:00


TypeError: expected string or bytes-like object, got 'float'

In [172]:
2019-06-30 00:00:00
  Stock             X  X_1         Trade
0   HPE   8585.287574  0.0   8585.287574
1   AES  29181.672025  0.0  29181.672025
2   KHC   6553.920605  0.0   6553.920605
3     C   1471.613321  0.0   1471.613321
4   HPQ  11361.424243  0.0  11361.424243
0    True
1    True
2    True
3    True
4    True
Name: Trade, dtype: bool
RangeIndex(start=0, stop=5, step=1)
RangeIndex(start=0, stop=5, step=1)

,Stock,Weight
0,CNC,0.242868
1,C,0.165852
2,AES,0.286748
3,KMI,0.049641
4,F,0.254891
